In [1]:
import json

In [2]:
with open('config/data-params.json', 'r') as fh:
    data_params = json.load(fh)

In [3]:
from src.etl import *

In [4]:
getData(**data_params)

(array([[ 59,  43,  50, ..., 140,  84,  72],
        [154, 126, 105, ..., 139, 142, 144],
        [255, 253, 253, ...,  83,  83,  84],
        ...,
        [ 35,  40,  42, ...,  77,  66,  50],
        [189, 186, 185, ..., 169, 171, 171],
        [229, 236, 234, ..., 173, 162, 161]], dtype=uint8),
 array([6, 9, 9, ..., 9, 1, 1]),
 array([[158, 159, 165, ..., 124, 129, 110],
        [235, 231, 232, ..., 178, 191, 199],
        [158, 158, 139, ...,   8,   3,   7],
        ...,
        [ 20,  19,  15, ...,  50,  53,  47],
        [ 25,  15,  23, ...,  80,  81,  80],
        [ 73,  98,  99, ...,  94,  58,  26]], dtype=uint8),
 array([3, 8, 8, ..., 5, 1, 7]))

In [5]:
train_data, train_label, test_data, test_label = getData(**data_params)

In [6]:
with open('config/model-params.json', 'r') as fh:
    model_params = json.load(fh)

In [7]:
from src.classifier import *

In [8]:
clf = clf_build(train_data, train_label, "RandomForestClassifier")

In [9]:
clf_predict(clf, test_data, test_label, "data/out/intial_preds.csv")

              precision    recall  f1-score   support

           0       0.54      0.56      0.55      1000
           1       0.52      0.54      0.53      1000
           2       0.37      0.33      0.35      1000
           3       0.35      0.29      0.32      1000
           4       0.39      0.39      0.39      1000
           5       0.42      0.39      0.40      1000
           6       0.47      0.56      0.51      1000
           7       0.51      0.45      0.47      1000
           8       0.58      0.61      0.59      1000
           9       0.48      0.55      0.51      1000

    accuracy                           0.47     10000
   macro avg       0.46      0.47      0.46     10000
weighted avg       0.46      0.47      0.46     10000



array([3, 8, 8, ..., 5, 5, 7])

In [10]:
from src.color_transform import *

In [11]:
X_train_gray = grayscale(train_data, 'X_train_gray')
X_test_gray = grayscale(test_data, 'X_test_gray')

In [12]:
X_test_gray.shape

(10000, 3072)

In [13]:
clf_predict(clf, X_test_gray, test_label, "data/out/grayscale_preds.csv")

              precision    recall  f1-score   support

           0       0.39      0.40      0.39      1000
           1       0.39      0.51      0.44      1000
           2       0.29      0.34      0.31      1000
           3       0.27      0.28      0.27      1000
           4       0.33      0.29      0.31      1000
           5       0.35      0.39      0.37      1000
           6       0.44      0.24      0.31      1000
           7       0.52      0.31      0.39      1000
           8       0.43      0.49      0.46      1000
           9       0.42      0.50      0.46      1000

    accuracy                           0.38     10000
   macro avg       0.38      0.38      0.37     10000
weighted avg       0.38      0.38      0.37     10000



array([2, 8, 8, ..., 3, 5, 7])

In [14]:
X_test_gray.shape

(10000, 3072)

In [15]:
sample1 = np.random.randint(50000, size = 5000)
sample2 = np.random.randint(50000, size = 5000)

In [16]:
Xt = train_data[sample1]
Xs = X_train_gray[sample2]

In [17]:
Xt.shape

(5000, 3072)

In [18]:
import ot

In [19]:
 # EMDTransport
ot_emd = ot.da.EMDTransport()
ot_emd.fit(Xs=Xs, Xt=Xt)

# SinkhornTransport
ot_sinkhorn = ot.da.SinkhornTransport(reg_e=1e-1)
ot_sinkhorn.fit(Xs=Xs, Xt=Xt)

c:\users\bill chen\appdata\local\programs\python\python38\lib\site-packages\ot\bregman.py:517: UserWarning: Sinkhorn did not converge. You might want to increase the number of iterations `numItermax` or the regularization parameter `reg`.
  warnings.warn("Sinkhorn did not converge. You might want to "


In [20]:
X_test_ot = ot_emd.transform(X_test_gray)

In [21]:
X_test_gray

array([[118, 118, 123, ...,  66,  74,  58],
       [234, 230, 231, ..., 177, 190, 195],
       [184, 181, 161, ...,   5,   4,   7],
       ...,
       [ 16,  15,  13, ...,  26,  27,  24],
       [ 32,  25,  32, ..., 109, 109, 107],
       [ 76, 102, 104, ..., 112,  65,  26]])

In [22]:
X_test_ot

array([[231., 230., 236., ..., 183., 191., 184.],
       [ 59.,  53.,  47., ...,  97., 108., 114.],
       [137., 133., 111., ..., 149., 166., 170.],
       ...,
       [ 21.,  21.,  18., ...,   0.,   1.,   2.],
       [ 85.,  81., 104., ...,  74.,  75.,  66.],
       [ 24.,  59.,  75., ...,  21.,  28., -32.]])

In [23]:
clf_predict(clf, X_test_ot, test_label, "data/out/ot_preds.csv")

              precision    recall  f1-score   support

           0       0.12      0.09      0.11      1000
           1       0.16      0.25      0.20      1000
           2       0.11      0.07      0.08      1000
           3       0.13      0.16      0.15      1000
           4       0.13      0.08      0.10      1000
           5       0.14      0.19      0.16      1000
           6       0.15      0.15      0.15      1000
           7       0.14      0.11      0.12      1000
           8       0.13      0.09      0.11      1000
           9       0.13      0.18      0.16      1000

    accuracy                           0.14     10000
   macro avg       0.13      0.14      0.13     10000
weighted avg       0.13      0.14      0.13     10000



array([8, 5, 1, ..., 5, 4, 6])

In [24]:
X_test_ot = ot_sinkhorn.transform(X_test_gray)

In [26]:
test_data

array([[158, 159, 165, ..., 124, 129, 110],
       [235, 231, 232, ..., 178, 191, 199],
       [158, 158, 139, ...,   8,   3,   7],
       ...,
       [ 20,  19,  15, ...,  50,  53,  47],
       [ 25,  15,  23, ...,  80,  81,  80],
       [ 73,  98,  99, ...,  94,  58,  26]], dtype=uint8)

In [27]:
X_test_ot

array([[ 99.91419482, 102.53791896, 109.34526314, ...,  44.50866223,
         52.18478428,  40.13832573],
       [177.07102043, 171.57569286, 171.36466427, ..., 198.29325619,
        207.55953129, 212.85831302],
       [153.22426184, 145.69960053, 121.3872964 , ...,  81.25718806,
         88.03215711,  97.08957515],
       ...,
       [ 83.01601243,  82.64213095,  80.07430433, ...,  70.66539863,
         71.37579723,  72.95171452],
       [ 83.68845909,  74.4111461 ,  81.98605099, ...,  94.91278141,
         92.54358023,  88.506044  ],
       [ 91.26685285, 113.87199121, 109.53617096, ...,  82.29664532,
         66.99965502,  21.97362248]])